# Population and region data linkage example for Deepti
Carl Higgs, 1 August 2022

This is an example of linking up UN World Urbanisation Prospects population data with the associated data sources file to get ISO3 country code, allowing further linkage for country level metadata, specifically continent or region grouping.  Then the top 5 cities in each tertile of population by continent are exported to a CSV file, with a basic set of data ('Region Name','Country or area','Urban Agglomeration',2022 population estimate,'Tertile').

The caveat to this is, it contains an urban agglomeration name and population estimate, but not its area, which means these are counts not densities.

Care must be taken if you were to associate an area to these population estimates, because the estimates are contingent on how that urban agglomeration was defined, and its not obvious from the data.   

Rather than link on an area and assume it is accurate, I would do something like the below but using the European Commission Global Human Settlements Urban Centres Database (or Funcational Urban Areas database, whichever is considered more appropriate).  This would be more straightforward as it already contains region groupings, population estimates and area, so all the preparatory linkage wouldn't be required.  However, the population estimate would be for 2015.  That could be defensible on two counts though 1) the estimate was made in 2019, which is more recent than the UN WUP modelling (2018), and may be more accurate (eg if more recent census data for that time were available for some locations); 2) relatively speaking, the most dense places in 2015 could broadly be expected to be the most dense places in 2022, regardless of absolute number of population.


In [1]:
import pandas as pd

## Analysis using UN World Urbanization Prospects population data
(for tertiles of population, not density)

In [2]:
# Get UN population data

# 5-year grouping data
#url = 'https://population.un.org/wup/Download/Files/WUP2018-F12-Cities_Over_300K.xls'

# annual data (to get 2022 projection)
url = 'https://population.un.org/wup/Download/Files/WUP2018-F22-Cities_Over_300K_Annual.xls'
df_population = pd.read_excel(url,'Data',header=16)
df_population

,Index,Country Code,Country or area,City Code,Urban Agglomeration,Note,Latitude,Longitude,1950,1951,...,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,1,4,Afghanistan,20001,Herat,NaN,34.348170,62.199670,82.468,83.114,...,781.380,809.800,838.394,867.422,897.041,927.393,958.549,990.635,1023.636,1057.573
1,2,4,Afghanistan,20002,Kabul,NaN,34.528887,69.172460,170.784,179.779,...,5034.106,5199.155,5371.550,5551.063,5737.138,5929.531,6127.953,6332.756,6543.594,6760.500
2,3,4,Afghanistan,20003,Kandahar,NaN,31.613320,65.710130,82.199,83.663,...,595.853,615.480,635.944,657.230,679.278,702.066,725.563,749.814,774.779,800.461
3,4,4,Afghanistan,20004,Mazar-e Sharif,NaN,36.709040,67.110870,30.000,31.308,...,708.798,735.572,762.162,788.909,816.040,843.747,872.137,901.348,931.381,962.262
4,5,8,Albania,20005,Tiranë (Tirana),NaN,41.327500,19.818890,84.513,88.866,...,542.740,549.219,555.132,560.491,565.301,569.577,573.332,576.588,579.349,581.626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,1856,894,Zambia,23277,Lusaka,NaN,-15.413374,28.277148,31.169,34.691,...,3621.323,3775.863,3934.683,4098.231,4266.517,4439.680,4617.689,4800.978,4989.296,5182.669
1856,1857,894,Zambia,23279,Ndola,NaN,-12.958670,28.636590,30.621,33.382,...,652.141,677.512,704.477,732.866,762.477,793.182,824.875,857.570,891.191,925.726
1857,1858,716,Zimbabwe,22510,Bulawayo,NaN,-20.150000,28.583330,91.635,92.200,...,682.123,697.347,714.397,733.107,753.306,774.894,797.799,822.049,847.603,874.479
1858,1859,716,Zimbabwe,22511,Chitungwiza,NaN,-18.012740,31.075550,38.645,40.487,...,428.988,439.293,450.483,462.539,475.422,489.115,503.603,518.923,535.060,552.027


In [3]:
# Get UN population linkage data
url = 'https://population.un.org/wup/Download/Files/WUP2018-DataSource-UrbanAgglomeration-and-CapitalCities.xls'
df_wup_datasources = pd.read_excel(url,'Data',header=16)
df_wup_datasources

,Index,Country or area,Country Code,ISO3 Alpha Code,ISO2 Alpha Code,City Code,Location name,Location type,Data,Sources,Statistical Concept,Note
0,1,Afghanistan,4,AFG,AF,20001,Herat,Non-Capital,Urban agglomerations and capital cities,"Census of 1979; Estimates for 1950, 1966, 1971...",City Proper,NaN
1,2,Afghanistan,4,AFG,AF,20002,Kabul,Capital,Urban agglomerations and capital cities,"Census of 1979; Estimates for 1966, 1971, 1986...",City Proper,NaN
2,3,Afghanistan,4,AFG,AF,20003,Kandahar,Non-Capital,Urban agglomerations and capital cities,"Census of 1979; Estimates for 1950, 1966, 1971...",City Proper,NaN
3,4,Afghanistan,4,AFG,AF,20004,Mazar-e Sharif,Non-Capital,Urban agglomerations and capital cities,"Census of 1979; Estimates for 1950, 2004 and 2...",City Proper,NaN
4,5,Albania,8,ALB,AL,20005,Tiranë (Tirana),Capital,Urban agglomerations and capital cities,"Censuses of 1950, 1955, 1960, 1969, 1979, 1989...",City Proper,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1940,1941,Zambia,894,ZMB,ZM,23277,Lusaka,Capital,Urban agglomerations and capital cities,"Censuses of 1950, 1963, 1969, 1980, 1990, 2000...",Urban Agglomeration,NaN
1941,1942,Zambia,894,ZMB,ZM,23279,Ndola,Non-Capital,Urban agglomerations and capital cities,"Censuses of 1963, 1969, 1980, 1990, 2000 and 2...",City Proper,NaN
1942,1943,Zimbabwe,716,ZWE,ZW,22510,Bulawayo,Non-Capital,Urban agglomerations and capital cities,"Censuses of 1951, 1956, 1962, 1973, 1982, 1992...",City Proper,NaN
1943,1944,Zimbabwe,716,ZWE,ZW,22511,Chitungwiza,Non-Capital,Urban agglomerations and capital cities,"Censuses of 1982, 1992, 2002 and 2012.",City Proper,NaN


In [4]:
# get UN country metadata
# retrieved from 'https://unstats.un.org/unsd/methodology/m49/overview/#', but had to manually select 'CSV'
df_region = pd.read_csv('./UNSD/UNSD — Methodology.csv', delimiter=";")
df_region

,Global Code,Global Name,Region Code,Region Name,Sub-region Code,Sub-region Name,Intermediate Region Code,Intermediate Region Name,Country or Area,M49 Code,ISO-alpha2 Code,ISO-alpha3 Code,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
0,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Algeria,12,DZ,DZA,NaN,NaN,NaN
1,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Egypt,818,EG,EGY,NaN,NaN,NaN
2,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Libya,434,LY,LBY,NaN,NaN,NaN
3,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Morocco,504,MA,MAR,NaN,NaN,NaN
4,1,World,2.0,Africa,15.0,Northern Africa,NaN,NaN,Sudan,729,SD,SDN,x,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1,World,9.0,Oceania,61.0,Polynesia,NaN,NaN,Samoa,882,WS,WSM,NaN,NaN,x
245,1,World,9.0,Oceania,61.0,Polynesia,NaN,NaN,Tokelau,772,TK,TKL,NaN,NaN,NaN
246,1,World,9.0,Oceania,61.0,Polynesia,NaN,NaN,Tonga,776,TO,TON,NaN,NaN,x
247,1,World,9.0,Oceania,61.0,Polynesia,NaN,NaN,Tuvalu,798,TV,TUV,x,NaN,x


In [5]:
# Note the columns of interest in each dataset
print("UN WUP Population data columns:")
print(list(df_population.columns))

print("\nUN WUP data sources (metadata for region code):")
print(list(df_wup_datasources.columns))

print("\nUNSTATS M49 metadata:")
print(list(df_region.columns))

UN WUP Population data columns:
['Index', 'Country Code', 'Country or area', 'City Code', 'Urban Agglomeration', 'Note', 'Latitude', 'Longitude', 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035]

UN WUP data sources (metadata for region code):
['Index', 'Country or area', 'Country Code', 'ISO3 Alpha Code', 'ISO2 Alpha Code', 'City Code', 'Location name', 'Location type', 'Data', 'Sources', 'Statistical Concept', 'Note']

UNSTATS M49 metadata:
['Global Code', 'Global Name', 'Region Code', 'Region Name', 'Sub-region Code', 'Sub-r

In [6]:
# link Population data with country metadata to get region classification
df = df_population[['Country Code', 'Country or area', 'City Code', 'Urban Agglomeration', 2022]]\
     .merge(df_wup_datasources[['City Code','ISO3 Alpha Code']],
           on='City Code')\
     .merge(df_region[['ISO-alpha3 Code', 'Region Name', 'Sub-region Name', 'Least Developed Countries (LDC)', 'Land Locked Developing Countries (LLDC)', 'Small Island Developing States (SIDS)']], 
           left_on='ISO3 Alpha Code',
           right_on='ISO-alpha3 Code')

df

,Country Code,Country or area,City Code,Urban Agglomeration,2022,ISO3 Alpha Code,ISO-alpha3 Code,Region Name,Sub-region Name,Least Developed Countries (LDC),Land Locked Developing Countries (LLDC),Small Island Developing States (SIDS)
0,4,Afghanistan,20001,Herat,665.477,AFG,AFG,Asia,Southern Asia,x,x,NaN
1,4,Afghanistan,20002,Kabul,4457.882,AFG,AFG,Asia,Southern Asia,x,x,NaN
2,4,Afghanistan,20003,Kandahar,526.775,AFG,AFG,Asia,Southern Asia,x,x,NaN
3,4,Afghanistan,20004,Mazar-e Sharif,594.551,AFG,AFG,Asia,Southern Asia,x,x,NaN
4,8,Albania,20005,Tiranë (Tirana),511.559,ALB,ALB,Europe,Southern Europe,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1847,894,Zambia,23277,Lusaka,3041.789,ZMB,ZMB,Africa,Sub-Saharan Africa,x,x,NaN
1848,894,Zambia,23279,Ndola,570.567,ZMB,ZMB,Africa,Sub-Saharan Africa,x,x,NaN
1849,716,Zimbabwe,22510,Bulawayo,643.427,ZWE,ZWE,Africa,Sub-Saharan Africa,NaN,x,NaN
1850,716,Zimbabwe,22511,Chitungwiza,397.149,ZWE,ZWE,Africa,Sub-Saharan Africa,NaN,x,NaN


In [7]:

df['region_population_tertile'] = df.groupby(['Region Name'])[2022].transform(
                     lambda x: pd.qcut(x, 3, labels=['Q1','Q2','Q3'])
)



In [24]:
df_final_wup = df.sort_values(['Region Name','region_population_tertile',2022],ascending=False)\
    .groupby(['Region Name','region_population_tertile'])\
    .head(5)[['Region Name','Country or area','Urban Agglomeration',2022,'region_population_tertile',2022]]

df_final_wup  
df_final_wup.to_csv('UN_city_population_tertiles_by_region.csv', encoding='utf-8')

# Analysis of GHS UCDB data
(for tertiles of population density)

In [11]:
# Get the GHS UCDB data
# zip file including multiple formats from
# https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_STAT_UCDB2015MT_GLOBE_R2019A/V1-2/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.zip
df_ghs = pd.read_csv('./GHS/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.csv')
df_ghs

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P90,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX
0,1,1,185,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,...,327680.831522,397443.031445,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41
1,2,2,42,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,...,0.000000,0.000000,0.000000,NaN,NaN,missing,NaN,0.128,0.284119,NaN
2,3,1,55,34.858517,-120.475511,34.989334,-120.389183,34.923123,-120.434372,United States,...,0.000000,0.000000,0.000000,0.000000,0.0,available,2.79174,0.48114,0.040129,23.64
3,4,1,48,36.582997,-121.952215,36.635743,-121.811816,36.607720,-121.882378,United States,...,0.000000,0.000000,0.000000,0.000000,0.0,available,NaN,0.44484,0.138683,42.17
4,5,1,60,34.388220,-119.853855,34.457831,-119.658413,34.427664,-119.743693,United States,...,0.000000,0.000000,0.000000,0.000000,0.0,available,4.25502,0.55676,0.061348,36.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13130,13131,1,70,-37.737746,176.104748,-37.631690,176.321892,-37.684080,176.183663,New Zealand,...,53105.795383,65009.255923,84583.724382,0.200739,6.0,available,14.05370,0.23856,0.409358,54.71
13131,13132,0,3,-6.759222,155.679978,-6.743003,155.703239,-6.749761,155.693009,Papua New Guinea,...,0.000000,0.000000,0.000000,0.888458,8.0,available,1.81968,4.1398,1.000000,NaN
13132,13133,1,23,-9.454775,159.911539,-9.422255,160.006750,-9.436571,159.963101,Solomon Islands,...,33751.399216,47752.046463,73669.684235,0.906212,8.0,available,1.90593,1.7473,0.824672,82.30
13133,13134,1,27,-22.304278,166.418428,-22.245962,166.483047,-22.277587,166.452136,New Caledonia,...,50409.868759,69251.056091,70631.796295,0.078658,4.0,available,NaN,0.25381,0.080976,55.15


In [12]:
print(list(df_ghs.columns))

['ID_HDC_G0', 'QA2_1V', 'AREA', 'BBX_LATMN', 'BBX_LONMN', 'BBX_LATMX', 'BBX_LONMX', 'GCPNT_LAT', 'GCPNT_LON', 'CTR_MN_NM', 'CTR_MN_ISO', 'XBRDR', 'XCTR_NBR', 'XC_NM_LST', 'XC_ISO_LST', 'GRGN_L1', 'GRGN_L2', 'UC_NM_MN', 'UC_NM_LST', 'UC_NM_SRC', 'H75_NBR', 'H90_NBR', 'H00_NBR', 'H75_AREA', 'H90_AREA', 'H00_AREA', 'E_BM_NM_LST', 'E_SL_LST', 'EL_AV_ALS', 'E_KG_NM_LST', 'E_RB_NM_LST', 'E_WR_P_90', 'E_WR_P_00', 'E_WR_P_14', 'E_WR_T_90', 'E_WR_T_00', 'E_WR_T_14', 'B75', 'B90', 'B00', 'B15', 'P75', 'P90', 'P00', 'P15', 'BUCAP75', 'BUCAP90', 'BUCAP00', 'BUCAP15', 'NTL_AV', 'GDP90_SM', 'GDP00_SM', 'GDP15_SM', 'INCM_CMI', 'DEV_CMI', 'TT2CC', 'E_GR_AV90', 'E_GR_AV00', 'E_GR_AV14', 'E_GR_AH90', 'E_GR_AM90', 'E_GR_AL90', 'E_GR_AT90', 'E_GR_AH00', 'E_GR_AM00', 'E_GR_AL00', 'E_GR_AT00', 'E_GR_AH14', 'E_GR_AM14', 'E_GR_AL14', 'E_GR_AT14', 'E_EC2E_E75', 'E_EC2E_E90', 'E_EC2E_E00', 'E_EC2E_E15', 'E_EC2E_R75', 'E_EC2E_R90', 'E_EC2E_R00', 'E_EC2E_R15', 'E_EC2E_I75', 'E_EC2E_I90', 'E_EC2E_I00', 'E_EC2E_I15

In [13]:
# Exclude urban centres that are 1) lacking names, and 2) smaller built up area of 1 sqkm or less
# I am assuming that those exclusion criteria are sensible ---
# I added them to avoid inclusion of very small, highly dense but not necessarily locations 
# which had been identified as urban centres (eg without a name you won't link w/ mortality data)
# You might want to consider including a minimum population estimate or modifying minimum area
df_ghs = df_ghs.loc[(df_ghs['UC_NM_MN'].isna()==False)&(df_ghs['B15']>1)].copy()
df_ghs

# (13135 - 9332 = 3,803 records excluded)

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P90,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX
0,1,1,185,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,...,327680.831522,397443.031445,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41
1,2,2,42,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,...,0.000000,0.000000,0.000000,NaN,NaN,missing,NaN,0.128,0.284119,NaN
2,3,1,55,34.858517,-120.475511,34.989334,-120.389183,34.923123,-120.434372,United States,...,0.000000,0.000000,0.000000,0.000000,0.0,available,2.79174,0.48114,0.040129,23.64
3,4,1,48,36.582997,-121.952215,36.635743,-121.811816,36.607720,-121.882378,United States,...,0.000000,0.000000,0.000000,0.000000,0.0,available,NaN,0.44484,0.138683,42.17
4,5,1,60,34.388220,-119.853855,34.457831,-119.658413,34.427664,-119.743693,United States,...,0.000000,0.000000,0.000000,0.000000,0.0,available,4.25502,0.55676,0.061348,36.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13129,13130,1,115,-36.838060,174.664782,-36.688502,174.830553,-36.771147,174.744125,New Zealand,...,9504.260010,11334.405762,14143.237518,0.091408,4.0,available,NaN,0.36038,0.583791,50.67
13130,13131,1,70,-37.737746,176.104748,-37.631690,176.321892,-37.684080,176.183663,New Zealand,...,53105.795383,65009.255923,84583.724382,0.200739,6.0,available,14.05370,0.23856,0.409358,54.71
13132,13133,1,23,-9.454775,159.911539,-9.422255,160.006750,-9.436571,159.963101,Solomon Islands,...,33751.399216,47752.046463,73669.684235,0.906212,8.0,available,1.90593,1.7473,0.824672,82.30
13133,13134,1,27,-22.304278,166.418428,-22.245962,166.483047,-22.277587,166.452136,New Caledonia,...,50409.868759,69251.056091,70631.796295,0.078658,4.0,available,NaN,0.25381,0.080976,55.15


In [14]:
# calculate population per sqkm of built up area in 2015
df_ghs['population_density_2015'] = df_ghs['P15']/df_ghs['B15']
df_ghs

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,population_density_2015
0,1,1,185,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,...,397443.031445,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41,6359.210748
1,2,2,42,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,...,0.000000,0.000000,NaN,NaN,missing,NaN,0.128,0.284119,NaN,6314.661586
2,3,1,55,34.858517,-120.475511,34.989334,-120.389183,34.923123,-120.434372,United States,...,0.000000,0.000000,0.000000,0.0,available,2.79174,0.48114,0.040129,23.64,2932.831529
3,4,1,48,36.582997,-121.952215,36.635743,-121.811816,36.607720,-121.882378,United States,...,0.000000,0.000000,0.000000,0.0,available,NaN,0.44484,0.138683,42.17,2441.411486
4,5,1,60,34.388220,-119.853855,34.457831,-119.658413,34.427664,-119.743693,United States,...,0.000000,0.000000,0.000000,0.0,available,4.25502,0.55676,0.061348,36.50,3011.755416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13129,13130,1,115,-36.838060,174.664782,-36.688502,174.830553,-36.771147,174.744125,New Zealand,...,11334.405762,14143.237518,0.091408,4.0,available,NaN,0.36038,0.583791,50.67,3861.378420
13130,13131,1,70,-37.737746,176.104748,-37.631690,176.321892,-37.684080,176.183663,New Zealand,...,65009.255923,84583.724382,0.200739,6.0,available,14.05370,0.23856,0.409358,54.71,2668.471856
13132,13133,1,23,-9.454775,159.911539,-9.422255,160.006750,-9.436571,159.963101,Solomon Islands,...,47752.046463,73669.684235,0.906212,8.0,available,1.90593,1.7473,0.824672,82.30,18102.255748
13133,13134,1,27,-22.304278,166.418428,-22.245962,166.483047,-22.277587,166.452136,New Caledonia,...,69251.056091,70631.796295,0.078658,4.0,available,NaN,0.25381,0.080976,55.15,5831.666991


In [15]:

df_ghs['region_density_tertile'] = df_ghs.groupby(['GRGN_L1'])['population_density_2015'].transform(
                     lambda x: pd.qcut(x, 3, labels=['Q1','Q2','Q3'])
)
df_ghs


,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,...,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX,population_density_2015,region_density_tertile
0,1,1,185,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,...,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41,6359.210748,Q3
1,2,2,42,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,...,0.000000,NaN,NaN,missing,NaN,0.128,0.284119,NaN,6314.661586,Q3
2,3,1,55,34.858517,-120.475511,34.989334,-120.389183,34.923123,-120.434372,United States,...,0.000000,0.000000,0.0,available,2.79174,0.48114,0.040129,23.64,2932.831529,Q3
3,4,1,48,36.582997,-121.952215,36.635743,-121.811816,36.607720,-121.882378,United States,...,0.000000,0.000000,0.0,available,NaN,0.44484,0.138683,42.17,2441.411486,Q2
4,5,1,60,34.388220,-119.853855,34.457831,-119.658413,34.427664,-119.743693,United States,...,0.000000,0.000000,0.0,available,4.25502,0.55676,0.061348,36.50,3011.755416,Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13129,13130,1,115,-36.838060,174.664782,-36.688502,174.830553,-36.771147,174.744125,New Zealand,...,14143.237518,0.091408,4.0,available,NaN,0.36038,0.583791,50.67,3861.378420,Q3
13130,13131,1,70,-37.737746,176.104748,-37.631690,176.321892,-37.684080,176.183663,New Zealand,...,84583.724382,0.200739,6.0,available,14.05370,0.23856,0.409358,54.71,2668.471856,Q2
13132,13133,1,23,-9.454775,159.911539,-9.422255,160.006750,-9.436571,159.963101,Solomon Islands,...,73669.684235,0.906212,8.0,available,1.90593,1.7473,0.824672,82.30,18102.255748,Q3
13133,13134,1,27,-22.304278,166.418428,-22.245962,166.483047,-22.277587,166.452136,New Caledonia,...,70631.796295,0.078658,4.0,available,NaN,0.25381,0.080976,55.15,5831.666991,Q3


In [16]:
df_final_ghs = df_ghs.sort_values(['GRGN_L1','region_density_tertile','population_density_2015'],ascending=False)\
    .groupby(['GRGN_L1','region_density_tertile'])\
    .head(5)[['GRGN_L1','CTR_MN_NM','UC_NM_MN','AREA','B15','P15','population_density_2015','region_density_tertile']]

df_final_ghs  

,GRGN_L1,CTR_MN_NM,UC_NM_MN,AREA,B15,P15,population_density_2015,region_density_tertile
13118,Oceania,Papua New Guinea,Madang,14,1.715542,110610.182449,64475.358286,Q3
13127,Oceania,Papua New Guinea,Kokopo,18,1.160540,69403.226654,59802.529485,Q3
13121,Oceania,Papua New Guinea,Lae,16,2.727797,90606.219116,33215.897661,Q3
13119,Oceania,Papua New Guinea,Port Moresby,52,10.818352,262014.953512,24219.489223,Q3
13132,Oceania,Solomon Islands,Honiara,23,4.069641,73669.684235,18102.255748,Q3
...,...,...,...,...,...,...,...,...
4491,Africa,Tanzania,Iwambi,16,4.577976,88201.596863,19266.503908,Q1
2395,Africa,Nigeria,Bida,33,14.572464,280560.307007,19252.770651,Q1
3624,Africa,Democratic Republic of the Congo,Kisangani,70,34.642719,666843.325867,19249.162304,Q1
3944,Africa,Democratic Republic of the Congo,Dungu,17,6.438098,123720.753662,19216.971439,Q1


In [22]:
# see the assumptions above with GHS data export, this may not be what you're after either
df_final_ghs.to_csv('GHS - city_population_density_tertiles_by_region.csv', encoding='utf-8')

In [21]:
print(list(df_final_ghs['GRGN_L1'].unique()))

['Oceania', 'Northern America', 'Latin America and the Caribbean', 'Europe', 'Asia', 'Africa']
